In [40]:
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Veri temizleme sırasında bazı çöp veriler ek 3 sütuna taştığı için o sütunları adlandırıp, sildik

In [41]:
csv_file_path = './earthquakedata.csv'
df = pd.read_csv(csv_file_path, sep=',', names=['Tarih', 'Saat', 'Enlem(N)', 'Boylam(E)', 'Derinlik(km)', 'MD', 'ML', 'Mw', 'Yer'])
df[['Tarih', 'Saat', 'Enlem(N)', 'Boylam(E)', 'Derinlik(km)', 'MD', 'ML', 'Mw', 'Yer','sil1','sil2','sil3']] = df['Tarih'].str.split(',', expand=True)

# Yeni bir CSV dosyasına yaz
yeni_csv_file_path = 'cleanedearthquake.csv'
df.to_csv(yeni_csv_file_path, index=False)


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Tarih         501 non-null    object
 1   Saat          501 non-null    object
 2   Enlem(N)      501 non-null    object
 3   Boylam(E)     501 non-null    object
 4   Derinlik(km)  501 non-null    object
 5   MD            501 non-null    object
 6   ML            501 non-null    object
 7   Mw            501 non-null    object
 8   Yer           501 non-null    object
 9   sil1          129 non-null    object
 10  sil2          98 non-null     object
 11  sil3          6 non-null      object
dtypes: object(12)
memory usage: 47.1+ KB


In [43]:
df = df.drop(['Enlem(N)','Boylam(E)','MD','Mw','sil1','sil2','sil3'], axis=1)

In [44]:
df.columns

Index(['Tarih', 'Saat', 'Derinlik(km)', 'ML', 'Yer'], dtype='object')

In [45]:
df.head(10)

,Tarih,Saat,Derinlik(km),ML,Yer
0,Tarih,Saat,Derinlik(km),ML,Yer
1,2023.12.08,20:27:48,7.9,1.8,BALABAN-DARENDE(MALATYA)
2,2023.12.08,20:00:27,2.9,2.0,POLAT-DOGANSEHIR(MALATYA)
3,2023.12.08,19:57:44,10.8,2.6,CUKURKAYA-DARENDE(MALATYA)
4,2023.12.08,19:48:15,7.8,1.5,SOGUT-DOGANSEHIR(MALATYA)
5,2023.12.08,19:39:35,9.0,1.1,GULOVA-KUMLU(HATAY)
6,2023.12.08,19:10:47,7.5,1.3,ASAGIULUPINAR-DARENDE(MALATYA)
7,2023.12.08,19:04:35,9.8,2.3,GEZIN-MADEN(ELAZIG)
8,2023.12.08,18:52:50,7.6,1.6,ASAGIULUPINAR-DARENDE(MALATYA)
9,2023.12.08,18:10:12,5.9,2.0,KOCALAR-(KAHRAMANMARAS)


Yer sütununda bulunan parantez dışındaki veriler çöp veriler olduğu için onları da siliyoruz.

In [46]:
df['Yer'] = df['Yer'].str.extract(r'\((.*?)\)')

In [47]:
df["Yer"]

0          NaN
1      MALATYA
2      MALATYA
3      MALATYA
4      MALATYA
        ...   
496      BURSA
497        NaN
498        NaN
499      BURSA
500      BURSA
Name: Yer, Length: 501, dtype: object

Yer sütunundaki boş değere sahip olan satırları siliyoruz.

In [48]:
df.dropna(subset=['Yer'], inplace=True)

In [49]:
df["Yer"]

1      MALATYA
2      MALATYA
3      MALATYA
4      MALATYA
5        HATAY
        ...   
492    MALATYA
494      SIVAS
496      BURSA
499      BURSA
500      BURSA
Name: Yer, Length: 371, dtype: object

Tarih ve saat sütunundaki verileri timestamp olarak birleştiriyoruz.

In [50]:
import datetime
import time

In [51]:
timestamp=[]
for d, t in zip(df['Tarih'], df['Saat']):
    try:
        ts = datetime.datetime.strptime(d + ' ' + t, '%Y.%m.%d %H:%M:%S')
        timestamp.append(time.mktime(ts.timetuple()))
    except ValueError as e:
        print(f"Error for date {d} and time {t}: {e}")
        timestamp.append('ValueError')

timeStamp = pd.Series(timestamp)
df['Timestamp'] = timeStamp.values

In [52]:
display(df)

,Tarih,Saat,Derinlik(km),ML,Yer,Timestamp
1,2023.12.08,20:27:48,7.9,1.8,MALATYA,1.702056e+09
2,2023.12.08,20:00:27,2.9,2.0,MALATYA,1.702055e+09
3,2023.12.08,19:57:44,10.8,2.6,MALATYA,1.702055e+09
4,2023.12.08,19:48:15,7.8,1.5,MALATYA,1.702054e+09
5,2023.12.08,19:39:35,9.0,1.1,HATAY,1.702054e+09
...,...,...,...,...,...,...
492,2023.12.04,13:33:36,4.1,1.8,MALATYA,1.701686e+09
494,2023.12.04,13:21:54,3.3,2.2,SIVAS,1.701685e+09
496,2023.12.04,13:13:57,5.0,1.0,BURSA,1.701685e+09
499,2023.12.04,12:57:25,7.2,1.3,BURSA,1.701684e+09


Artık "Tarih" ve "Saat" sütunu ile bir işimiz kalmadığı için onları da siliyoruz.

In [53]:
df = df.drop(['Tarih','Saat'], axis=1)

In [54]:
df=df.rename(columns = {'Derinlik(km)':'Derinlik'})

In [55]:
df["Derinlik"] = df.Derinlik.astype(float)
df["ML"] = df.ML.astype(float)

In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 371 entries, 1 to 500
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Derinlik   371 non-null    float64
 1   ML         371 non-null    float64
 2   Yer        371 non-null    object 
 3   Timestamp  371 non-null    float64
dtypes: float64(3), object(1)
memory usage: 14.5+ KB


In [58]:
df.to_csv(yeni_csv_file_path, index=False)